# Polymofismus

Polymorfismus je vlastnost programovacího jazyka vytvářet kódy, které jsou použitelné pro objekty několika různých typů (v OOP tříd). Podpora polymorfismu je klíčovým rysem moderních programovacích jazyků, neboť jen s pomocí polymorfismu lze psát flexibilní a přitom stručné a přehledné programy. Je klíčový především pro objektově orientované jazyka, v nichž je vytváření tříd základním prostředkem tvorby komplexnější programů (OOP jazyky musí být polymorfní od narození).

Podívejme se na dva triviální příklady polymorfismu:

In [2]:
def first(collection): # polymorfní funkce vracející první prvek z kolekce
    return collection[0]

Tuto funkci lze zavolat na instance překvapivě velkého množství tříd.

In [4]:
first([2,3,5]) #seznamu

2

In [5]:
first([(2,3,5)]) # n-tice

(2, 3, 5)

In [6]:
first("Sílor") #řetězce

'S'

A mnoha dalších (zkuste nějaké ještě najít).

Tento kód však není absolutně polymorfní tj. nelze jej využít pro objekty libovolné třídy.

In [8]:
first(2) # číslo nemá žádné podprvky a proto nemá smysl vrátit první

TypeError: 'int' object is not subscriptable

In [9]:
first({1,2,3}) # to už je trochu překvapivější, ale ani množina nemá první prvek

TypeError: 'set' object is not subscriptable

Absolutně polymorfní kód je relativně těžké napsat (objekty mají jen málo věcí univerzálně sdílených). Metoda `identical` testuje, zda jsou dva objekty identické, Využívá funkci `id`, která vrací jednoznačný číselný identifikátor každého objektu (typicky je to adresa objektu v paměti).

In [10]:
def identical(a,b):
    return id(a) == id(b)

In [11]:
identical(2, 2)

True

In [12]:
identical("a", "a")

True

In [13]:
identical([],[]) # ale dva prázdné seznamy nejsou identické

False

> **Poznámka:** Namísto porovnání číslených identifikátorů lze samozřejmě využít **polymorfní** operátor `is`.

Programovací jazyky poskytují různé mechanismy pro podporu polymorfirmu. Python nabízí tři z nich:
1. automatické (implicitní) přetypování (v Pythonu relativně okrajový mechanismus využívaný v Pythonu jen v případě přetypování mezi čísly)
2. duck typing (dynamické typování) — hlavní prostředek polymorfismu v Pythonu
3. dědičnost (dědičnost nabízí polymorfismus jako důsledek specifičtějšího mechanismu)

## Duck typing

*Duck typing* (český překlad *kachní typování* se zatím neujal) vychází z následující úvahy:

```
If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck.
```

Autorem je americký básník *James Whitcomb Riley* a ukazuje přirozený přístup logického uvažování. Na to abychom poznali kachnu, tak není zapotřebí, aby byla opatřena visačkou `kachna` stačí jen, pokud se chová jako kachna (přesněji jen v námi požadovaném kontextu, věta například nepožaduje, abychom zkoumali její rozmnožování, což je ve skutečnosti to, podle čeho o příslušnosti jedince k druhu rozhoduje biologie).

> **Poznámka**: Detailnější diskusi tohoto tzv. kachního testu najdete na anglické Wikipedii (https://en.wikipedia.org/wiki/Duck_test).
>
> Pro hlubší pochopení problematiky doporučuji hlavně dvě parafráze:
>
>```
>If it looks like a duck, and quacks like a duck, we have at least to consider the possibility that we have a small aquatic bird of the family Anatidae on our hands.
>```
>Douglas Adams
>
>```
>If it looks like a terrorist, if it acts like a terrorist, if it walks like a terrorist, if it fights like a terrorist, it's a terrorist, right?
>```
>Сергей Викторович Лавров

V kontextu programovacích jazyků je důsledkem aplikace "kachního" přístupu, důraz na to jak se objekt chová a nikoliv na to, jaké je konkrétní třídy.

V naší funkci `first` klademe na objekt jediné dva požadavky:
1. podporuje indexaci (od nuly), tj. speciální metodu `__index__`
2. má alespoň jeden prvek (to většina typových systémů nedokáže popsat, takže se tento aspekt nezohledňuje)

Nikde se neuvádí jméno třídy tohoto objektu, a je to nejen zbytečné, ale i kontraproduktivní. Pokud uděláme rybníček pro kachny, pak je výhodné, pokud ho mohou používat i husy. Podobně pokud si uděláme řidičák pro auta, je rozumné ho uznávat i pro malé motocykly.

Někdy bývá dokonce složité pojmenovat skupinu tříd, pro něž je daný polymorfní kód určen. Většinou se používá:
1. obecný název, pokud je k dispozici (zde například by to byla [neprázdná] *sekvence*)
2. označení typického zástupce se zevšeobecňující příponou (*list-like*, česky snad seznamovité objekty)
3. označení podle funkce (*indexable* respektove *indexovatelné*)


Obecně lze říci, že dva objekty lze používat polymorfně (tj. lze napsat polymorfní kód, který pracuje s oběma objekty), pokud oba mají metodu, která:
1. má stejný identifikátor
2. má kompatibilní parametry (definice kompatibility parametrů je v Pythonu dosti komplexní; v zásadě lze říct, jsou kompatibilní, pokud obě metody přijímají požadované skutečné parametry)
3. má kompatibilní návratovou hodnotu (pokud je využívána, pak obě metody musí vracet objekt stejného typu nebo objekty, které lze v daném kontextu zaměňovat(tj. nemusí být nutně stejného typu)
4. splňují tzv. kontrakt tj.
    1. stejně interpretují vstupní parametry
    2. vracejí požadované výstupní hodnoty 
    3. požadovaným způsobem modifikují (resp. nemodifikují) zůčastněné objekty (klíčový je objekt `self`)
    4. požadovaným způsobem interagují s okolím (např. s operačním systémem, uživatelem, vzdálenou službou)
    5. vyhazují požadované výjimky

Z těchto požadavků je snadno popsatelný jen první požadavek a také je nejsnadněji detekovatelný. Naopak čtvrtý požadavek, se často specifikuje je obtížně a také se jeho splnění špatně detekuje (běžně vyžaduje vytvoření pomocného kódu pro tzv. `unit testing`)

Podívejme se na jednoduchý příklad. Předpokládejme objekt, který representuje nějakou metriku. Metrika je objekt (v matematickém pojetí) funkce, která definuje vzdálenost mezi dvěma body.

Nejdříve si definujme detailní požadavky na objekty metrik (všimněte si, že se vyhýbám slovu třída).

1. mají metodu distance
2. tato metoda přijímá dva parametry, jimiž jsou dvojice dvou reálných čísel
3. vrací reálné číslo
4. a splňující tento kontrakt
   1. vstupní hodnoty se interpretují jako kartézské souřadnice bodů (ve stejné soustavě souřadnic) 
   2. výsledkem je vzdálenost (ta musí splňovat pravidla pro metriky 
   viz https://cs.wikipedia.org/wiki/Metrick%C3%BD_prostor#Definice
   3. volání nemodifikuje objekt `self` (tj. zjištění vzdálenosti dvou bodů nemá vliv na výpočet 
   vzdálenosti ostatních), parametry se přirozeně také nemění (což je ale u čísel v Pythonu standardem)
   4. nevzniká žádná výjimka (výjimky na pozadí, vznikající například při nedostatku paměti se nepočítají) 

Uveďme ukázkovou implementaci třídy, jejíž objekty splňují výše uvedené požadavky. Realizovaná metrika, je klasická euklidovská (https://mathworld.wolfram.com/EuclideanMetric.html):

In [4]:
import math

class EuclideanMetric:
    def distance(self, p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [6]:
# základní test funkčnosti

m = EuclideanMetric()
print(m.distance((1.0, 1.0), (2.0, 2.0)))

1.4142135623730951


Pro ukázku polymorfismu musíme vytvořit ještě jednu třídu, jejíž objekty splňují naše požadavky- Zvolíme tzv. Manhattanskou distanci (

In [10]:
class TaxicabMetric:
    def distance(self, p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        return abs(x1-x2) + abs(y1-y2)

In [11]:
m = TaxicabMetric()
print(m.distance((1.0, 1.0), (2.0, 2.0)))

2.0


> **Úkol**: Prostudujte Manhattanskou metriku (metriku). Proč je v některých případech využívána namísto euklidovské?

Nyní již můžeme psát polymorfní kódy. Například následující funkce zjistí, jaký ze dvou bodů je bližší počátečnímu bodu a to v dané metrice.

In [14]:
def nearest(start, a, b, metric):
    return a if metric.distance(start, a) < metric.distance(start, b) else b

print(nearest((0,0), (1,2), (-2,3), EuclideanMetric()))

(1, 2)


Tento kód je polymorfní, neboť dokáže pracovat s objekty různých tříd. V případě parametru `metric` jsou to objekty obou tříd splňující naše požadavky.

In [15]:
print(nearest((0.0, 0.0), (1.2,2.0), (-2.0,3.0), TaxicabMetric()))

(1, 2)


Navíc to může být potenciálně i objekt i jiných prozatím ještě neexistujících tříd. Tato *otevřenost* je typická pro většinu mechanismů polymorfismu.  Dalším typickým rysem pythonského polymorfismu je dynamický charekter. O tom, jaká verze metody `distance` se rozhodne až při vykonávání kódu (nikoliv při jeho překladu).

> **Úkol**: Definujte třídu pro Chebyševovu metriku (viz https://en.wikipedia.org/wiki/Chebyshev_distance) a  vyzkoušejte, zda funguje s naší polymorfní funkcí.

> **Poznámka**: Funkce `nearest` je samozřejmě polymorfní i v ostatních parametrech. Body lze například zadávat jako seznnam celých čísel. Ve skutečnosti, je v Pythonu obtížné napsat kód, který není polymorfní.

> **Úkol**: Popište, co musí splňovat objekty, který musí být parametrem následující funkce (především ten první označený `seq`). Uveďte několik tříd, jejichž objekty, tyto požadavky splňují.

In [16]:
def in_zone(seq, minval, maxval):
    for value in seq:
        if not (minval <= value  <= maxval):
            return False
    return True

Příklad využití:

In [19]:
in_zone([2,1,3,6], 0, 5)

False

Parametr `seq` musí umožnovat iteraci (postupné procházení), přičemž každá vrácená položka musí umožňovat vzájemné porovnání s hodnotami `mainval` a `maxval`. Na ty se jinak nekladou žádné další požadavky. Obecně (i když nikoliv zcela přesně) jsou to iterátory přes hodnoty s definovaným uspořádáním.

Příklady (sami vyzkoušejte):
* seznam celých čísel (hodnoty `minval` a `maxval` by měly být číselné), vrací zda jsou všechny čísla v intervalu
* řetězec (hodnoty `minval` a `maxval` by měly být jednoznakové řetězce), vrací zda všechny znaky v Unicode sadě mezi těmito znaky
* textový proud (získaný např. voláním `open`, vrací zda jsou všechny řádky lexikograficky mezi řetězci `minval` a `maxval`)

Dynamický polymorfismus v Pythonu není *nominální* (což je typická vlastnost *duck typingu*). To znamená, že nezáleží na nejakém formálním označení či sdílené značce (*tagu*) sdílené na úrovni tříd (tj.třídy např. nemusí mít společný prefix v názvu, nemusí mít nějaký společný metatribut apod.)

Příklad ze života: Pokud vyhlásíme fotbalový turnaj, kterého se mohou zúčastnit všechny týmy, které souhlasí s pravidly (a podepíší to při jeho zahájení) a mlčky předpokládáme, že umí hrát fotbal, pak to není nominální polymorfismus. Pokud však vyžadujeme např. aby byly členy nějaké organizace (což je atribut týmu) nebo působili v nějakém městě, pak už se jedná o nominální polymorfismus (týmy musí něco nominálně splňovat).

Tento přístup je velmi flexibilné avšak na druhou stranu komplikuje zjištění, zda je nějaký objekt skutečně splňuje požadavky příslušného polymorfního kódu. Jedinou možností, jak to ověřit, je objekt použít. Musíme však být připraveni, že (v lepším případě) vznikne nějaká neočekávaná výjimka (například výjimka signalizující, že objekt případnou metodu nemá) v horším případě to vede k nedefinovanému chování (podivné hodnoty).

### Abstraktní bázové třídy

Python tento problém řeší pomocí tzv. abstraktních bázových tříd (*abstract base class* zkratka *ABC*). To je nepovinný prostředek jak nominálně označkovat, že daná třída splňuje určitý protokol tj. množinu metod. 

Abstraktní bázové třídy tvoří hierachii, kdy každá třída může rozšiřovat jinou abstraktní třídu (označováno jako `inheritance`, ale má to jen velmi málo společného s dědičností uvedenou v další kapitole). V tomto případě musí objekt splňující danou abstraktní třídu splňovat i protokol třídy, kteroz rožšiřuje (a tak rekurzivně dál).

Prostředek se ve standardní knihovně omezuje v zásadě jen na tři typy objektů:
* čísla (vytváří základní hierarchii čísel, viz modul `numbers` a jeho dokumentace https://docs.python.org/3.8/library/numbers.html)
* kolekce (vytváří hierarchii různých obecných druhů kolekcí, viz modul`collections.abc` https://docs.python.org/3/library/collections.abc.html)
* proudy (viz https://docs.python.org/3/library/io.html#class-hierarchy)

Jako příklad se podívejme na klíčovou abstraktní třídu `collections.abc.Sequence`.

Každý objekt, který se chce prohlásit za nemodifikovatelnou sekvenci (a být plnohodnotně používán v polymorfním kódu pracujícím nad sekvencemi) musí:
1. Implementovat speciální metodu `__getitem__`, která zajišťuje podporu indexace 
2. implementovány metod dvou dalších abstraktních tříd a to `Reversible` a `Collection`

Třída `collections.abc.Reversible` vyžaduje implementaci reverzního iterátoru (je získán voláním metody `reversed` a prochází sekvenci od posledního prvku k prvnímu), což je zajímavý rys typický pouze pro Python.

Třída `collections.abc.Collection` ve skutečnosti nevyžaduje žádné specifické metody, neboť metody uvedené v sloupci `Abstract methods` jen opakují metody vyžadované třemi triviálními abstraktními třídami:

* `__contains__`: testovaní, zda je prvek obsažen v kontejneru, využíváno operátorem `in` (převzato ze třídy `collections.abc.Container`)
* `__len__`: zjištění počtu prvků (využíváno funkcí `len`), převzato ze třídy `collections.abc.Sized` (český ekvivalent mne nenapadá, snad *Počitatelné*)
* `__iter__`: vrací běžný (dopředný) iterátor, je tak zřejmé, že sekvence jsou tzv iterovatelné tj. lze je procházet prvek po prvku (implementují správným způsobem  speciální metodu `__iter_` a tak splňují abstraktní třídu collections.abc.Iterable).`

> **Úkol**: Ověřte, že n-tice splňuje rozhraní (protokol) abstraktní třídy sekvence.

In [5]:
t = (1,2,3)
print(len(t)) # je počitatelná
print(2 in t) # je to kontejner, v němž lze procházet
print(list(iter(t))) # funguje iterátor (je jím naplněn seznam)
print(list(reversed(t))) # funguje reversní iterátor
t[0] # a funguje indexace

3
True
[1, 2, 3]
[3, 2, 1]


1

Formální (nominální) ověření, že objekt splňuje všechny požadavky abstraktní třídy (překladač však přirozeně nic nekontroluje). Nejjednodušší je využití vestavěné funkce `isinstance`, která ověřuje, zda je objekt instancí dané třídy. Kromě třídy, ze které objekt vznikl, zohledňuje i složitější případy včetně nominální implementace dané třídy (a také dědičnost, k níž se dostaneme za chvíli).

In [6]:
from collections.abc import Sequence
isinstance(t, Sequence)

True

Mechanismus abstraktních tříd hraje v Pythonu jen pomocnou roli a v rámci duck typingu není využívána (tj. lze stačí reálná shoda protokolu, abstraktní bázové třídy nejsou vůbec brány v potaz). Mnozí u

I přes tato omezení má reálná využití:
* inspekce typů používaná především v ladícím testovacím kódu
* pomoc při tvorbě bezpečného polymorfního kódu resp. při tvorbě tříd splňujících daný protokol (dokumentace určuje příslušné metody a často i jejich kontrakt) a je-li použit mechanismus dědičnosti, pak i implicitní implementace některých metod (viz níže)
* při (nepovinné, ale často užitečné) statické typové anotaci kódu (dodatečná informace například pro editory, které tak mohou lépe doplňovat syntaxi)

Typové anostace se využívají dodatečný zdroj informací pro některé nástroje a dokumentaci, překladač Pythonu ji však nikdy nevyužívá! (tj, program s anotacemi, či bez nich bude fungovat stejně)

In [7]:
from collections.abc import MutableSequence

def multiappend(seq: MutableSequence, item, howmany : int) -> MutableSequence:
    """
    Append item howmany-times to mutable sequence.
    
    Returns:
        changed sequence `seq`
    """
    for _ in range(howmany):
        seq.append(item)
    return seq

Klíčové je především určení typu prvního parametru, Může to být libovolný objekt implementující třídu `collections.abc.MutableSequence`. Díky tomu např. *Jupyter Lab* ví, že má nabídnout metodu `append` po zapsaní `seq.` a stisku klávesy `Tab`. Druhý parametr typován není (může být libovolný). Třetí parametr je typován konkrétním typem (nabízí se typování typem `number.Integral`, ale to je trochu nadbytečné, neboť Python podporuje jen jednu třídu splňující toto rozhraní.)





## Dědičnost

Dědičnost (*inheritance*) je standardní mechanismus, který umožňuje vytvářet nové třídy rozšiřováním existujících, a to:
* *skládáním jejich datových členů (atributů) s možností přidání dalších*
* *převzetím jejich metod, přidáním dalších a modifikací převzatých*
* *podporu dynamického polymorfismu* (prostřednictvím odkazů na objekty)

Tento mechanismus existoval už v nejstarším objektově orientovaném jazyce [Simula 68](https://en.wikipedia.org/wiki/Simula) a je součástí valné většiny objektově orientovaných jazyků. Je tak součástí standardního objektově orientovaného paradigmatu a to tím nejkontraverznějším, neboť byl a stále je různě chápán resp. dokonce nechápán.

Důvodem je skutečnost, že je tento mechanismus využíván pro různé účely, které na sobě do určité míry závisejí, ale mohou být v jiné podobě nabízeny i bez využití mechanismu dědičnosti (např. polymorfismus či skládání, ale i sdílení implementací) a to často bez nepříjemných postranních efektů, které jejich souběh v dědičnosti přináší.

Následující přehled uvádí přehled klíčových účelů resp. konceptů, pro které je dědičnost využívána (a opět je nutno říci, že často i zneužívána).

1. subtyping resp. vytváření podtypů
   1. vytváření hierarchií typů (representace 
   2. podpora univerzální třídy *Any* (všechny třídy jsou přímo či nepřímo odvozeny od jediné třídy)
   3. podpora univerzální podtřídy *Nothing* (objekt je instancí všech tříd)
   4. podpora polymorfismu (abstraktní třídy)
2. sdílení chování/implementací metod mezi objekty různých tříd (znovupoužití)
   1. mixiny
   2. metatřídy
   3. delegování (zřetězení) metod
3. strukturální skládání atributů resp. datových členů (vytváření komplexnějších objekt, nikdy není základní)
   1. části si navenek zachovávají identitu (podpora polymorfismu)
   2. části jsou zapouzdřené (bez podpory polymorfismu)

Jrdnotlivé OOP jazyky podporují různé aspekty dědičnosti a kladou na ně různý důraz. Python podporuje v zásadě všechny aspekty dědičnosti (kromě univerzální podtřídy), žádný z nich však není pro Python klíčový a mnohé jsou ve skutečnosti zcela okrajové např. univerzální nadtřída, mixiny, metatřídy apod. (dědičnost obecně hraje v Pythonu relativně malou roli).

> **Upozornění**: Role dědičnosti se i přes základní shodu, může v jednotlivých OOP jazycích podstatně lišit. Je to jedna z věcí, které znesnadňují přechod mezi jazyky a mohou být velmi matoucí (snadno si můžete přenést návyky, které jsou v kontextu jazyky vysloveně špatné). Navic se role dědičnosti v multiparadigmatických jazycích (kam do značné míry spadá i Python) může lišit i mezi jednotlivými frameworky.


### Základní mechanismus dědičnosti

Základní mechanismus dědičnosti je v Pythonu relativně jednoduchý. Ukážeme si ho na jednoduchém (umělém) příkladě.

Nejdříve vytvoříme základní (bázovou) třídu representující dvojici hodnot (Python podporuje obecné n-tice a tak je tato třída nadbytečná). Její konstruktor přrjímá dva parametry a ukládá je do atributů. Dále je definována metoda, která oba parametry prohodí (tu standardní n-tice nemá, proč?) a funkci `print`, která vypíše obsah (ani ta není obsažena v rozhraní n-tice, opět proč?)

In [2]:
class TupleBox: # 
    def __init__(self, p1, p2): # konstruktor se dvěma parametry
        self.a1 = p1  # vytvoření a inicializace dvou atributů
        self.a2 = p2
    def change_attrs(self): # prohazuje oba atributy
        self.a1, self.a2 = self.a2, self.a1
    def print(self): # vypisuje textovou representaci do standardního výstupu
        print(f"item 1: {self.a1}, item 2: {self.a2}")

Vyzkoušíme vytvořit instanci.

In [3]:
a = TupleBox(1,2)
a.change_attrs()
a.print()

item 1: 2, item 2: 1


A nyní ze třídy odvodíme novou třídu `NamedTupleBox`, která je 
* **rozšířením**  třídy `TupleBox` (tj. její instance umí všechno, co instance třídy `TupleBox` a ještě něco navíc)
* **specializací** třídy `TupleBox` tj. její instance jsou zároveň instancemi třídy `TupleBox` (v této interpretaci však jsou dodatečné metody opravdu navíc). To mimo jiné znamená, že jakákoliv kód schopný
pracovat s objekty třídy `TupleBox` je automaticky polymorfní, neboť dokáže pracovat s objekty dvou tříd!

In [16]:
class NamedTupleBox(TupleBox): # základní třída je v závorkách
    def __init__(self, p1, p2, name):
        super().__init__(p1, p2) # voláme kosntruktor předka (delegování)
        self.name = name # nový atribut (instance třídy B mají jméno)
    def  print(self):
                print(f"({self.name}) item 1: {self.a1}, item 2: {self.a2}")
    def myName(self):
        return self.name

Tato třída:
1. volá v rámci svého konstruktoru konstruktor základní třídy a tak každá její instance obsahuje stejné atributy jako instance bázové třídy. Volání se děje prostřednictvím funkce `super`, která vrací  odkaz na stejný objekt jako `self` avšak s typem základní třídy (nikoliv třídy odvozené). Pomocí tohoto odkazu tak můžeme zavolat původní (nepředefinované) verze metod včetně původního konstruktoru. 
2. přidává vlastní atribut `name`
3. dědí metodu `change_attrs`, která pracuje s atributy vytvořenými konstruktorem základní třídy
4. předefinovává metodu `print`. Pokud jí zavoláme na instanci odvozené třídy, tak sice vykoná stejnou akci jako u třídy bázové (tj. vypíše svou textovou representaci na standardní výstup), ale mírně ji modifikuje (insatance má své vlastní jméno a tak ho vypíše). Metoda pracuje s původními atributy a nově přidaným atributem.
5. přidává metodu `maName`, která vrátí jméno instance (tato metoda pracuje jen s přidaným atributem)

Na rozhraní obou tříd se podíváme detailněji. Nejdříve vytvoříme objekty obou tříd.

In [17]:
baseObject = TupleBox(1,2)
extendedObject = NamedTupleBox(2,3, "the best of object")

Původní atributy `a1` a `a2` podporují oba objekty:

In [6]:
print(baseObject.a1)
print(baseObject.a2)

print(extendedObject.a1)
print(extendedObject.a2)

1
2
2
3


Atribut `name` však má jen objekt odvozené (rozšířené) třídy:

In [8]:
print(extendedObject.name)

print(baseObject.name)

the best of object


AttributeError: 'TupleBox' object has no attribute 'name'

U obou objektů lze atributy prohodit.

In [10]:
baseObject.change_attrs()
extendedObject.change_attrs()

A oba také mají motodu `print`. Její provedení u odvozené třídy se však liší.

In [11]:
baseObject.print()
extendedObject.print()

item 1: 2, item 2: 1
(the best of object) item 1: 3, item 2: 2


Rozšířená třída také navíc podporuje metodu `myName`.

In [18]:
extendedObject.myName()

'the best of object'

Vyzkoušíme ještě chování funkce `isinstance`:

In [20]:
isinstance(baseObject, TupleBox) # objekt vždy instancí své třídy

True

In [22]:
isinstance(extendedObject, NamedTupleBox)  # podobně i pro objekt rozšířené třídy

True

Jak však již bylo řečeno, platí, že rozšířený objekt je zároveň i instancí základní třídy.

In [24]:
isinstance(extendedObject, TupleBox)

True

Opačně to ovšem neplatí! Je to zřejmé, neboť objekt základí třídy postrádá metodu `myName`.

In [27]:
isinstance(baseObject, NamedTupleBox)

False

Skutečnost, že objekt rozšířené (odvozené) třídy je zároveň instancí třídy základní a může ho vždy zastoupit je jedním ze základních rysů dědičnosti a je znám pod dvěma označeními:

1. relace `is-a` mezi třídami. Pojmenovaná n-tice (tj. instance třídy *NamedTupleBox*) je (*is a*)  n-ticí.
To se podstatně liší od relace `has a`, kterou se popisuje kompozice skládání. "Auto je dopravní prostředek" tj. vztah mezi objekty dopravních prostředků a objekty aut **lze representovat** dědičností (třída dopravních prostředků je základní třída, třída aut je třída odvozená). Naproti lze říci, že „auto má motor“ (nikoliv „auto je motor“ nebo „motor je auto“)
2. Liskovové princip zastoupení (*Liskov substitution principle*). Je to formalizovaná a zcela striktní verze výše uvedeného požadavku: *Nechť $\phi(x)$ je vlastnost prokazatelná objektu $x$ typu $T$. Potom $\phi(y)$ je pravdivé pro všechny objekty $y$ typu $S$, kde $S$ je odvozeno z $T$ (resp. je rozšířením $T$).*

Princip zastoupení se v praxi nepoužívá k formálnímu důkazu (je těžké formálně specifikovat všechny vlastnosti objektů dané třídy) ale k formálnímu či praktickému testování.

Musí mimo jiné platit, že
* metody odvozené třídy musí přijímat všechny hodnoty, které jsou přípustné v předefinovávané  metodě základní třídy (může však přijímat i další navíc)
* metody odvozené třídy musí vracet hodnoty jen z množiny přípustných hodnot předefinovávané metody základní třídy (může však vracet jen podmnožinu těchto hodnot)
* metoda odvozené třídy může vyvolávat jen ty výjimky, které vyvolává předefinovávané metody (opět samozřejmě nemusí vyvolívat všechny), pokud v nějaké situacii vznikne v původní metodě výjimka, pak v odvozené metodě může (ale nemusí) vzniknout. Naopak nevznikne-li, pak mesmí vzniknout ani v předefinované.

Všimněte si, že tyto podmínky odpovídají kontraktu pro metody polymorfně zaměnitelných objektů. V Pythonu je **primární polymorfismus založený na kontraktech**. Dva objekty mohou být zastupitelné i v případě, že mezi nimi neexistuje vztah dědičnosti. Jediný rozdíl je v tom, že:
1. dědičnost by měla zajistit úplnou zastupitelnost (tj. pro všechny metody z rozhraní základní třídy poskytuje i třída odvozená, a to příslibem splnění kontraktu)
2. zděděné metody splňují kontrakt automaticky (jsou totiž identické)
3. schopnost objektu splnit daný kontrakt lze ověřit ověřením, zda je daný objekt instancí příslušné základní třídy pomocí funkce `instanceof` (v případě obecného polymorfismu to lze zajistit prostřednictvím mechanismu
abstraktních bázových tříd, to je však podporováno jen u těch nejdůležitějších protokolů)

>**Poznámka k názvosloví:**
>
>Pro **základní třídu** se používá i termín **bázová třída** (což je přímočařejší překlad z anglického *base >class*) resp. **nadtřída** (množina objektů základní třídy je *nadmnožinou* objektů třídy rozšířující). >Možný je i termín **předek** vycházející z uspořádání, které relace dědičnosti představuje (obecně však >tento termín nedoporučuji je příliš svázán s relací mezi konkrétními objekty (tvrzení typu „Adam je předkem >Jákoba“).
>
>Pro **odvozenou** či **rozšířenou třídu** lze naopak použít termíny **podtřída** nebo **potomek**.

>**Úkol**: Diskutujte vhodnost použití dědičnosti pro representaci následující vztahů mezi potenciálními třídami (při hodnocení záleží na kontextu použití, proto uvažujte navržené kontexty, resp. přidejte další).
>
> * Šelma → Pes (prodejna chovatelských potřeb a krmiva, české dráhy, zoolog)
> * Zaměstnanec → Sekretářka (firemní IS, pracovní úřad)
> * Komplexní číslo → Racionální číslo (algebraický systém)
> * Auto → Nákladní auto (policie, spediční firma)
> * Auto → Červené auto (prodejce aut, pojišťovna, policie)

Relace dědičnosti může být použita i tranzitivně tj. lze vytvářet celé hierarchie tříd od nejobecnějších po nejspecifičtější. V jedné z fází vývoje objektově orientovaného paradigmatu byly v módě rozsáhlé a především hluboké hierarchie. V praxi se však ukázalo, že takto pojeté využití dědičnosti je neflexibilní a obtížně spravovatelné (dědičnost může výrazně narušit zapouzdření a tím zvyšovat závislosti mezi třídami).

V současnosti se tak preferují jen relativně mělké hierararchie o dvou až pěti úrovních. V Pythonu se kromě hierarchie dané využitím univerzální základní třídy dědičnost omezuje jen na izolované hierarchie v rámci některých standardních knihoven (ve skutečnosti jsem si nemohl na žádný konkrétní příklad kromě tříd výjimek vzpomenout).

Poněkud jiná situace je u některých knihoven zapouzdřující hierarchii v jiných jazycích a frameworcích (typicky GUI), následující obrázek ukazuje zjednodušenou hierarchii v knihovně *Qt* dostupné v Pythonu prostřednictvím knihovny [PyQt](https://riverbankcomputing.com/software/pyqt/intro).

![Qt class hierarchy](https://qt-wiki-uploads.s3.amazonaws.com/images/4/4c/Beginner-Class-Hierarchy.jpg)


### Univerzální základní třída `object`

Všechny třídy. které namají uvedenou bázovou třídu, se v Pythonu automaticky odvozují od třídy `object`. Všechny třídy jsou tak odvozené ze třídy `object` ať už přímo, či nepřímo.
Tato skutečnost však v Pythonu nehraje téměř žádnou roli (v Pythonu 3, v Pythonu 2 byla situace jiná).

* objekt není klíčový pro dosažení univerzálního polymorfismu (opačně je tomu Javě)
* nehraje roli v obecné podpoře různých druhů polymorfismu (na rozdíl např. od Javy, kde souvisí s tzv. generiky)
* běžně se nevytvářejí instance třídy `object` (v Javascriptu je to běžné, neboť konstruktor univerzální třídy je základem OOP podpory)
*  instance třídy `object` nemůže nést žádné atributy a nemůže tak být využívána jako slovník, jehož klíči jsou identifikátory (opět na rozdíl např. od Javascriptu).

Jedinou funkcí třídy objekt tak zůstává implementace několika univerzálních speciálních metod:

In [29]:
o = object()

print(o) # využití metody __str__
print ( o == object()) # využití metody _eq__ 

False


Standardní verze metody vrací jméno třídy následované adresou objektu, standardní porovnání pak porovnává odkazy na objekty (tj. vrací `True` jsou-li objekty identické).

O tom, že jsou tyto speciální metody se můžeme přesvědčit tím, že vytvořímě odvozenou třídu, která nic nepřidává (žadné atributy či metody) ani nemodifukuje.

In [30]:
class AnotherObject:  # není nutné psát class AnotherObject(object):
    pass              # prázdné tělo je nutné representovat příkazem `pass`

In [31]:
o = AnotherObject()

print(o) # využití metody __str__
print ( o == object()) # využití metody _eq__ 

False


> **Úkol**: Zjistěte, jaké základní implementace (speciálních) metod nabizí třída `object`.

In [34]:
print(bool(o)) # každý objekt je standardně pravdivý (metoda __bool__)
# některé klíčové třídy tuto metodu předefinovávají 
print(bool("")) # ale některé se s tím nemohli smířit

True
False


V Pythonu lze předefinovat i standardní třídy. Implementujme například roztržitý seznam, který si zapamatuje průměrně jen polovinu položek, které do něj vložíme. Tato nepříliš užitečná třída se od běžného seznamu (zdánlivě) liší jen v jediné metodě — `append`.

V implementaci konstruktoru je využit speciální parametr, jenž je uvozen znakem `*`. Tento parametr přejímá všechny nadbytečné poziční parametry (tj. parametry neuložené do formálních parametrů, jež by byly uvedeny dříve).  Formálně lze tento parametr označit libovolným identifikátorem, ale v praxi se používá jen  jméno `args` (zkratka za *arguments*). 

Hodnotou tohoto parametru je seznam (může být i prázdný). V našem konstruktoru přejímá všechny poziční parametry a využívá je při volání konstruktoru základní třídy. I zde je použit zápis s prefixem `*`, který však má v případě skutečných parametrů opačnou funkci. Vezme příslušný seznam a jeho položky postupně předá jako poziční parametry (bez hvězdičky by se předal jen jeden parametr, jehož hodnotou by byl seznam).  Konstruktor nadtřídy tak získá stejné parametr, jako byly předány konstruktoru odvozené třídy.

In [2]:
import random

class DistractedList(list):
    def __init_(self, *args):
        super().__init__(*args) # volání konstruktoru nadtřídy
    def append(self, item):
        if random.random() > 0.5:
            super().append(item)

In [ ]:
Nově odvozenou (rozšířenou) třídu vyzkoušíme tím, že do vložíme čísla od 0 do 99.

In [3]:
d = DistractedList()

for i in range(100):
    d.append(i)
print(d)
print(len(d))

[0, 8, 10, 11, 12, 13, 19, 20, 21, 22, 26, 27, 28, 29, 30, 33, 34, 36, 37, 40, 46, 47, 48, 49, 54, 57, 60, 61, 65, 67, 68, 69, 70, 74, 75, 77, 78, 79, 80, 82, 85, 86, 87, 88, 89, 91, 93, 94, 95, 97, 99]
51


Vše se zdá v pořádku, ale není tomu tak. Na problémy narazíme v okamžiku, kdy se roztržitý seznam pokusíme naplnit pomocí iterátoru, předaného konstruktoru (což je přístup, který preferovala většina zkušenějších pythonistů).

In [7]:
d2 = DistractedList(range(100))
print(d2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


Je zřejmé, že se něco nepodařilo, protože seznam obsahuje všechny prvky. Je to trochu překvapivé, neboť by bylo lze předpokládat, že originální implementace konstruktoru bude hodnoty z předaného iterátoru vkládat pomocí metody `append` (která je v případě naší třídy předefinována).

Narážíme tak na jeden z hlavních problémů dědičnosti, neboť náš kód je závislý interní implementaci, která není součástí rozhraní a nemusí předvídat problémy spojené s použitím v odvozené třídě. Zde se autoři evidentně rozhodli nevyužít metodu `append` a to z důvodů efektivity (vestavěné kolekce často používají nízkoúrovňový kód v jazyce C, aby dosáhli rozumné efektivity).

Řešení v tomto případě existuje, i když není příliš elegantní. Stačí rozšířit předefinovanou verzi konstruktoru. Naštěstí tato funkce přijímá jen jediný nepovinný parametr.

In [25]:
class DistractedList(list):
    def __init__(self, it = None):
        super().__init__() # volání bezparametrického konstruktoru (vytváří prázdný seznam)
        if it is not None:
            for item in it:
                self.append(item)
    def append(self, item):
        if random.random() > 0.5:
            super().append(item)

In [26]:
d3 = DistractedList(range(100))
print(d3)

[0, 1, 2, 4, 6, 8, 9, 10, 13, 14, 16, 17, 22, 23, 24, 27, 28, 30, 32, 34, 35, 36, 40, 41, 42, 47, 48, 52, 53, 54, 56, 58, 60, 64, 67, 68, 69, 70, 71, 74, 78, 81, 86, 88, 89, 90, 93, 94, 95, 96, 97]


> **Úkol**: Předefinování metody `append` není dostatečné. Seznam podporuje i metodu `insert`. Implementujte třídu  `DistractedList` s předefinovanou metodou `insert` (s obdobnou sémentikou jako má `append`).

In [29]:
class DistractedList(list):
    def __init__(self, it = None):
        super().__init__() # volání bezparametrického konstruktoru (vytváří prázdný seznam)
        if it is not None:
            for item in it:
                self.append(item)
    def append(self, item):
        if random.random() > 0.5:
            super().append(item)
    def insert(self, index, item):
        if random.random() > 0.5:
            super().insert(index, item)
            
d4 = DistractedList()
d4.insert(0,1)
d4.insert(0,2)
d4.insert(0,3)

print(d4)

[3, 2]


> **Poznámka:** Náhodné vkládání na určité pozice může vést k téměř nepředvídatelnému chování (na rozdíl od `append` není zajištěno ani zachování pořadí). Bylo by proto vhodnější tuto metodu u roztržitého seznamu nepodporovat. Při dědění však nelze žádnou z metod odstranit, neboť by to narušovalo princip zastupitelnosti – instance odvozené třídy musí být z hlediska polymorfního kódu neodlišitelné od instancí základní třídy (ony to de facto jsou instance základní třídy i když nepřímo) a tak musí podporovat i všechny jejich metody (v opačném případě je snadno odlišíte).
> Na druhou stranu lze přijmout představu, že odmítnutí vložení může být někdy zcela adekvátní reakce, kterou sice standardní implementace sice nevyužívá, ale není nemyslitelná u specializovaných instancí. Při tomto pohledu je důsledné vyhazování výjimky `NotImplementedError` v odvozené metodě `insert` akceptovatelné.

> **Úkol**: Prostřednictvím dědičnosti implementujte slovník, který při použití indexace pro získání hodnoty vrací `None`, pokud použitý klíč neexistuje (standardně je vyvolána výjimka). Rada: uvnitř předefinované speciální metody `__getitem__` můžete využít metodu `get`.

### Ostatní využití dědičnosti

Python využívá dědičnost, i pro další účely:
1. dědění z abstraktních bázových tříd (klíčové je zde to, že u instancí odvozené třídy lze testovat splnění rozhraní, odvozená třída však musí dodat kód klíčových metod a atributy pro uložení dat)
2. odvozování specilizovaných tříd výjimek
3. skládání více objektů se zachováním rozhraní těchto objektů (s možností přidání společných metod). Zde se využívá tzv. vícenásobná dědičnost. Tento typ dědičnosti není podporován v některých z klíčových OOP jazyků (např. v Javě) a i v Pythonu je užíván zřídka (detailnější informace s hezkým příkladem viz https://www.python-course.eu/python3_multiple_inheritance.php)
4. přidávání určité funkčnosti do tříd, které ji nepodporují nebo ji podporují jen částečně (tzv. přimíšewní angl. *mixin*).

První a čtvrtou možnost si ukažeme na příkladu tzv. cyklického seznamu, což je nemodifikovatelný seznam, jenž se navenek tváří jako n-násobné zřetězení (opakování) kratšího seznamu. Ve skutečnosti však v sobě uchovává jen tento kratší seznam. Tato kolekce je kupodivu docela užitečná (pro representaci cyklicky se opakujících struktur, například cyklicky se opakujících se nastavení).

In [37]:
from collections.abc import Sequence

class CyclicList(Sequence):
    def __init__(self, base_iter, repetition):
        self.data = list(base_iter)
        self.rep = repetition
        self.sublen = len(self.data)
    def __len__(self): # délka seznamu
        return self.rep * self.sublen
    def __getitem__(self, index):
        if not (0 <= index < len(self)):
            raise IndexError("Index out of range")
        return self.data[index % self.sublen]

Odvození z abstraktní bázové třídy `Sequence` není v Pythonu nezbytné. Přináší však dvě výhody.
Za prvé lze formálně testovat, že instance splňuje rozhraní (protokol) sekvencí. Aby to však byla skutečně pravda musíme implementovat dvě speciální metody (`__len__` a `__getitem__`), které splňují určitý kontrakt:
* zjištění délky vrací pro daný objekt fixní a nezápornou celočíselnou hodnotu
* indexace (určená jen pro čtená) vrací pro daný index fixní hodnotu, index navíc musí ležet v přípustném rozhraní (0 až len - 1), jinak je vyhozena výjimka `IndexError`. 

Překladač nekontroluje, zda jsou příslušné metody definovány, tím spíše zda splňují kontrakt.

In [38]:
clist = CyclicList("abc", 10)
#ověříme zda je se třída hlásí k rozhraní sekvence 
print(isinstance(clist, Sequence))
# a nyní zda ji skutečně splňuje
print(len(clist))
for i in range(30): # vvypíšeme všechny prvky
    print(clist[i], end="")

True
30
abcabcabcabcabcabcabcabcabcabc

Otestovat je nutné i chybové stavy:

In [39]:
clist[30]

IndexError: Index out of range

Druhou výhodou je že abstraktní třída funguje jako *mixin* tj. přidala nám i další metody, které u sekvence předokládáme:

In [41]:
# je iterovatelná přes všechny prvky (metoda __iter__)
for c in clist:
    print(c, end="")
# lze ověřit zda obsahuje nějaký objekt (metoda __contains__)
print()
print("a" in clist)
print("z" in clist)

abcabcabcabcabcabcabcabcabcabc
True
False


In [43]:
# metodu index (hledá první výskyt)
print(clist.index("c"))
print(clist.index("d")) # nenajde-li vyhazuje výjimku

2


ValueError: 

In [44]:
# další přimíšenou metodou je `count`, která počítá výskyty
print(clist.count("a"))

10


A také reverzní iterátor:

In [45]:
for c in reversed(clist):
    print(c, end="")

cbacbacbacbacbacbacbacbacbacba

Tato funčnost ale zdarma není zadarmo. Implementace iterátorů je relativně efektivní, neboť využíví indexace (iterátor postupně vrací prvky [0], [1] až [len-1] resp. naopak.

Silně neefektivní je však implementace metod pro vyhledávání `__contains__`, `index` a `count`, neboť všechny tyto metody prohledávají celou n-krát opakovanou posloupnost (u prvních dvou jen v případě neúspěchu, u poslední metody vždy).

In [51]:
giantlist = CyclicList([0,1], 1_000_000)
# seznam zaujímá jen pár bytů, navenek však má 2 000 000 prvků
print(len(giantlist))

2000000


Vyhledání nuly je rychlé (stačí porovnat jen jeden prvek).

In [52]:
%%timeit
0 in giantlist

1.52 µs ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [53]:
%%timeit
2 in giantlist

1.99 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Je přitom jasné, že není potřeba prohledávat celé cyklicky opakované pole, stačí projít jen uložený podseznam. Proto je vhodnější předefinovat zděděné metody.

>**Úkol**: Vytvořte novoui definici třídy `CyclicList`, která je sice odvozena z abstraktní bázové třídy`Sequence`, ale předefinovává neefektivní metody (tím, že je deleguje na uložený podseznam).

In [54]:
class CyclicList(Sequence):
    def __init__(self, base_iter, repetition):
        self.data = list(base_iter)
        self.rep = repetition
        self.sublen = len(self.data)
    def __len__(self): # délka seznamu
        return self.rep * self.sublen
    def __getitem__(self, index):
        if not (0 <= index < len(self)):
            raise IndexError("Index out of range")
        return self.data[index % self.sublen]
    def __contains__(self, item):
        return item in self.data
    def index(self, item): # index je běžná metoda
        return self.data.index(item) # stačí vrátit index prvního výskytu
    def count(self, item):
        return self.rep * self.data.count(item)

Novou implementaci vyzkoušíme.

In [56]:
giantlist = CyclicList([0,1], 1_000_000)
# seznam zaujímá jen pár bytů, navenek však má 2 000 000 prvků
print(giantlist.count(0))
print(1 in giantlist)

1000000
True


In [58]:
%%timeit

0 in giantlist

225 ns ± 4.41 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Všimněte se, že nyní je vyhledávání dokonce ještě rychlejší než než původní (pozitivní) hledání (asi 8×). Důvodem je skutečnost, že volání zděděných metod vyžaduje jistou režii.